In [273]:
import pandas as pd
import numpy as np
import sys
import sklearn
import io
import random

In [274]:
dataset = "train2.csv"
testfile = "test2.csv"

In [275]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

#read dataframe from csv file:
df = pd.read_csv(dataset,header=None, names = col_names)
df_test = pd.read_csv(testfile, header=None, names = col_names)

#Size of dataframe:
print("Dataset: ", df.shape)
print("Testfile: ", df_test.shape)

#Debug 5 first item
df.head(5)

Dataset:  (88161, 42)
Testfile:  (37811, 42)


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal
1,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal
2,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal
3,0,tcp,private,REJ,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune
4,0,tcp,private,S0,0,0,0,0,0,0,...,9,0.04,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune


In [276]:
#Dem so label moi loai:
print("Dataset :", df["label"].value_counts())
print("Testfile: ", df_test["label"].value_counts())

Dataset : normal             47223
neptune            28796
ipsweep             2530
satan               2517
portsweep           2053
smurf               1837
nmap                1055
back                 664
teardrop             637
warezclient          607
pod                  137
guess_passwd          26
buffer_overflow       19
land                  12
warezmaster           12
loadmodule             8
imap                   8
ftp_write              6
rootkit                6
multihop               5
spy                    1
phf                    1
perl                   1
Name: label, dtype: int64
Testfile:  normal             20119
neptune            12418
satan               1116
ipsweep             1069
portsweep            878
smurf                809
nmap                 438
back                 292
warezclient          283
teardrop             255
pod                   64
guess_passwd          27
buffer_overflow       11
warezmaster            8
land                   6
roo

In [277]:
#Tim cac cot khong phai binary:

print("Chua la binary Train:")
for col_name in df.columns:
    if(df[col_name].dtypes == 'object' ):
        print(col_name)

print("Chua la binary Test: ")
for col_name in df_test.columns:
    if(df_test[col_name].dtypes == 'object'):
        print(col_name)


Chua la binary Train:
protocol_type
service
flag
label
Chua la binary Test: 
protocol_type
service
flag
label


**Label Encoder**

In [278]:
#Label encoder:

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

col_object_name = ['protocol_type',"service","flag"]

df_value = df[col_object_name]
df_test_value = df_test[col_object_name]

df_value.head()


,protocol_type,service,flag
0,udp,other,SF
1,tcp,http,SF
2,tcp,http,SF
3,tcp,private,REJ
4,tcp,private,S0


In [279]:
#protocol type:
unique_protocol = sorted(df.protocol_type.unique())
temp1 = 'Protocol_type_'
unique_protocol2 = [temp1 + x for x in unique_protocol]
print(unique_protocol2)


['Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp']


In [280]:
#Service:
unique_service = sorted(df.service.unique())

temp2 = 'Service_'
unique_service2 = [temp2 + x for x in unique_service]
print(unique_service2)

['Service_IRC', 'Service_X11', 'Service_Z39_50', 'Service_aol', 'Service_auth', 'Service_bgp', 'Service_courier', 'Service_csnet_ns', 'Service_ctf', 'Service_daytime', 'Service_discard', 'Service_domain', 'Service_domain_u', 'Service_echo', 'Service_eco_i', 'Service_ecr_i', 'Service_efs', 'Service_exec', 'Service_finger', 'Service_ftp', 'Service_ftp_data', 'Service_gopher', 'Service_harvest', 'Service_hostnames', 'Service_http', 'Service_http_443', 'Service_http_8001', 'Service_imap4', 'Service_iso_tsap', 'Service_klogin', 'Service_kshell', 'Service_ldap', 'Service_link', 'Service_login', 'Service_mtp', 'Service_name', 'Service_netbios_dgm', 'Service_netbios_ns', 'Service_netbios_ssn', 'Service_netstat', 'Service_nnsp', 'Service_nntp', 'Service_ntp_u', 'Service_other', 'Service_pm_dump', 'Service_pop_2', 'Service_pop_3', 'Service_printer', 'Service_private', 'Service_red_i', 'Service_remote_job', 'Service_rje', 'Service_shell', 'Service_smtp', 'Service_sql_net', 'Service_ssh', 'Service

In [281]:
# flag:
unique_flag = sorted(df.flag.unique())
temp3 = 'Flag_'
unique_flag2 = [temp3 + x for x in unique_flag]
print(unique_flag2)

['Flag_OTH', 'Flag_REJ', 'Flag_RSTO', 'Flag_RSTOS0', 'Flag_RSTR', 'Flag_S0', 'Flag_S1', 'Flag_S2', 'Flag_S3', 'Flag_SF', 'Flag_SH']


In [282]:

# put together
dumcols = unique_protocol2 + unique_service2 + unique_flag2
print(dumcols)


['Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp', 'Service_IRC', 'Service_X11', 'Service_Z39_50', 'Service_aol', 'Service_auth', 'Service_bgp', 'Service_courier', 'Service_csnet_ns', 'Service_ctf', 'Service_daytime', 'Service_discard', 'Service_domain', 'Service_domain_u', 'Service_echo', 'Service_eco_i', 'Service_ecr_i', 'Service_efs', 'Service_exec', 'Service_finger', 'Service_ftp', 'Service_ftp_data', 'Service_gopher', 'Service_harvest', 'Service_hostnames', 'Service_http', 'Service_http_443', 'Service_http_8001', 'Service_imap4', 'Service_iso_tsap', 'Service_klogin', 'Service_kshell', 'Service_ldap', 'Service_link', 'Service_login', 'Service_mtp', 'Service_name', 'Service_netbios_dgm', 'Service_netbios_ns', 'Service_netbios_ssn', 'Service_netstat', 'Service_nnsp', 'Service_nntp', 'Service_ntp_u', 'Service_other', 'Service_pm_dump', 'Service_pop_2', 'Service_pop_3', 'Service_printer', 'Service_private', 'Service_red_i', 'Service_remote_job', 'Service_rje', 'Service_sh

In [283]:
#test set service:
unique_service_test = sorted(df_test.service.unique())
temp_test_1 = 'Service_'
unique_service_test2 = [temp_test_1 + x for x in unique_service_test]
#test set flag:
unique_flag_test = sorted(df_test.flag.unique())
temp_test_2 = 'Flag_'
unique_flag_test2 = [temp_test_2 + x for x in unique_flag_test]
#test set protocol:
unique_protocol_test = sorted(df_test.protocol_type.unique())
temp_test_3 = 'Protocol_type_'
unique_protocol_test2 = [temp_test_3 + x for x in unique_protocol_test]

#fix service thieu du lieu tu train:
dumcols_test = unique_protocol_test2 + unique_service_test2 + unique_flag_test2
print(dumcols_test)


['Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp', 'Service_IRC', 'Service_X11', 'Service_Z39_50', 'Service_auth', 'Service_bgp', 'Service_courier', 'Service_csnet_ns', 'Service_ctf', 'Service_daytime', 'Service_discard', 'Service_domain', 'Service_domain_u', 'Service_echo', 'Service_eco_i', 'Service_ecr_i', 'Service_efs', 'Service_exec', 'Service_finger', 'Service_ftp', 'Service_ftp_data', 'Service_gopher', 'Service_harvest', 'Service_hostnames', 'Service_http', 'Service_http_2784', 'Service_http_443', 'Service_imap4', 'Service_iso_tsap', 'Service_klogin', 'Service_kshell', 'Service_ldap', 'Service_link', 'Service_login', 'Service_mtp', 'Service_name', 'Service_netbios_dgm', 'Service_netbios_ns', 'Service_netbios_ssn', 'Service_netstat', 'Service_nnsp', 'Service_nntp', 'Service_ntp_u', 'Service_other', 'Service_pm_dump', 'Service_pop_2', 'Service_pop_3', 'Service_printer', 'Service_private', 'Service_red_i', 'Service_remote_job', 'Service_rje', 'Service_shell', 'Service_

In [284]:
df_value_enc = df_value.apply(LabelEncoder().fit_transform)

print(df_value.head())
print("===========================")
print(df_value_enc.head())

df_test_value_enc = df_test_value.apply(LabelEncoder().fit_transform)


  protocol_type  service flag
0           udp    other   SF
1           tcp     http   SF
2           tcp     http   SF
3           tcp  private  REJ
4           tcp  private   S0
   protocol_type  service  flag
0              2       43     9
1              1       24     9
2              1       24     9
3              1       48     1
4              1       48     5


**OneHot Encoder**

In [285]:
#create OneHotEncoder object:
enc = OneHotEncoder(categories='auto')

#fit and transform df_value_enc:
df_value_enc_enc = enc.fit_transform(df_value_enc)
df_cat_data = pd.DataFrame(df_value_enc_enc.toarray(),columns=dumcols)

#fit and transform df_test_value_enc:
df_test_value_enc_enc = enc.fit_transform(df_test_value_enc)
df_test_cat_data = pd.DataFrame(df_test_value_enc_enc.toarray(),columns=dumcols_test)


df_cat_data.head()



,Protocol_type_icmp,Protocol_type_tcp,Protocol_type_udp,Service_IRC,Service_X11,Service_Z39_50,Service_aol,Service_auth,Service_bgp,Service_courier,...,Flag_REJ,Flag_RSTO,Flag_RSTOS0,Flag_RSTR,Flag_S0,Flag_S1,Flag_S2,Flag_S3,Flag_SF,Flag_SH
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [286]:
#check df_cat_data, df_test_cat_data:
print("===========================")
print(df_cat_data.shape)
print(df_test_cat_data.shape)

(88161, 83)
(37811, 82)


In [303]:
#Kiem tra su khac nhau giua cac hang theo tung cot cua 2 dataframe:
print(col_object_name)

for col_name in col_object_name:
    print(col_name)
    trainlist = df[col_name].to_list()
    testlist = df_test[col_name].to_list()
    difference = list(set(trainlist) ^ set(testlist))
    string = col_name + "_"
    difference = [string + x for x in difference]
    print(difference)
    


['protocol_type', 'service', 'flag']
protocol_type
[]
service
['service_http_2784', 'service_http_8001', 'service_aol']
flag
[]


In [305]:
#Chi co serivce bi thay doi:
trainservice = df.service.to_list()
testservice = df_test.service.to_list()
difference_service = list(set(trainservice) ^ set(testservice))
string = "Service_"
difference_service = [string + x for x in difference_service]
print(difference_service)

['Service_http_2784', 'Service_http_8001', 'Service_aol']


In [306]:
#them cac hang thieu vao test:
for col in difference_service:
    df_test_cat_data[col] = 0

print(df_cat_data.shape)
print(df_test_cat_data.shape)


(88161, 83)
(37811, 84)


In [307]:
#Them cac hang bi thieu vao train:
for col in difference_service:
    df_cat_data[col] = 0

print(df_cat_data.shape)
print(df_test_cat_data.shape)

(88161, 84)
(37811, 84)


In [308]:
#them cac truong da encode vao lai:
newdf = df.join(df_cat_data)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
newdf.drop('flag', axis=1, inplace=True)

newdf_test = df_test.join(df_test_cat_data)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
newdf_test.drop('flag', axis=1, inplace=True)

print(newdf.shape)
print(newdf_test.shape)


(88161, 123)
(37811, 123)


In [309]:
#So hoa label:
label_df = newdf['label']
label_df_test = newdf_test['label']

print(label_df.unique())
print(label_df_test.unique())

difference_label = list(set(label_df_test.unique()) ^ set(label_df.unique()))

print("===========================")
print(difference_label)

['normal' 'neptune' 'warezclient' 'portsweep' 'teardrop' 'nmap' 'satan'
 'pod' 'ipsweep' 'smurf' 'back' 'ftp_write' 'guess_passwd' 'multihop'
 'buffer_overflow' 'imap' 'warezmaster' 'rootkit' 'land' 'loadmodule'
 'spy' 'phf' 'perl']
['neptune' 'normal' 'ipsweep' 'portsweep' 'smurf' 'warezclient' 'satan'
 'teardrop' 'back' 'guess_passwd' 'nmap' 'rootkit' 'pod' 'phf'
 'buffer_overflow' 'warezmaster' 'imap' 'land' 'multihop' 'ftp_write'
 'spy' 'perl' 'loadmodule']
[]


In [310]:
#Them gia tri phan loai cho label:
label_dict = { 'normal' : 0, 
                'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2,
                'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4
                }

print(label_dict)

print("===========================")

newlabel_df = label_df.replace(label_dict)
newlabel_df_test = label_df_test.replace(label_dict)

newdf['label'] = newlabel_df
newdf_test['label'] = newlabel_df_test



{'normal': 0, 'neptune': 1, 'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1, 'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1, 'ipsweep': 2, 'nmap': 2, 'portsweep': 2, 'satan': 2, 'mscan': 2, 'saint': 2, 'ftp_write': 3, 'guess_passwd': 3, 'imap': 3, 'multihop': 3, 'phf': 3, 'spy': 3, 'warezclient': 3, 'warezmaster': 3, 'sendmail': 3, 'named': 3, 'snmpgetattack': 3, 'snmpguess': 3, 'xlock': 3, 'xsnoop': 3, 'httptunnel': 3, 'buffer_overflow': 4, 'loadmodule': 4, 'perl': 4, 'rootkit': 4, 'ps': 4, 'sqlattack': 4, 'xterm': 4}


*Phan loai kieu tan cong bang gia tri label*
![](attackclass.png)


In [311]:
#Khoang phan loai
is_DoS = [0,1]
is_Probe = [0,2]
is_R2L = [0,3]
is_U2R = [0,4]

DoS_df = newdf[newdf['label'].isin(is_DoS)]
Probe_df = newdf[newdf['label'].isin(is_Probe)]
R2L_df = newdf[newdf['label'].isin(is_R2L)]
U2R_df = newdf[newdf['label'].isin(is_U2R)]

#test
DoS_df_test = newdf_test[newdf_test['label'].isin(is_DoS)]
Probe_df_test = newdf_test[newdf_test['label'].isin(is_Probe)]
R2L_df_test = newdf_test[newdf_test['label'].isin(is_R2L)]
U2R_df_test = newdf_test[newdf_test['label'].isin(is_U2R)]

print("Train:")
print("DoS:", DoS_df.shape)
print("Probe:", Probe_df.shape)
print("R2L:", R2L_df.shape)
print("U2R:", U2R_df.shape)

print("Test:")
print("DoS:", DoS_df_test.shape)
print("Probe:", Probe_df_test.shape)
print("R2L:", R2L_df_test.shape)
print("U2R:", U2R_df_test.shape)


Train:
DoS: (79306, 123)
Probe: (55378, 123)
R2L: (47889, 123)
U2R: (47257, 123)
Test:
DoS: (33963, 123)
Probe: (23620, 123)
R2L: (20448, 123)
U2R: (20137, 123)


**Chuyen ve dang vector**

In [312]:
# X thuoc tinh, Y gia tri

#Train
X_DoS = DoS_df.drop('label', axis=1)
Y_DoS = DoS_df['label']

X_Probe = Probe_df.drop('label', axis=1)
Y_Probe = Probe_df['label']

X_R2L = R2L_df.drop('label', axis=1)
Y_R2L = R2L_df['label']

X_U2R = U2R_df.drop('label', axis=1)
Y_U2R = U2R_df['label']

#Test
X_DoS_test = DoS_df_test.drop('label', axis=1)
Y_DoS_test = DoS_df_test['label']

X_Probe_test = Probe_df_test.drop('label', axis=1)
Y_Probe_test = Probe_df_test['label']

X_R2L_test = R2L_df_test.drop('label', axis=1)
Y_R2L_test = R2L_df_test['label']

X_U2R_test = U2R_df_test.drop('label', axis=1)
Y_U2R_test = U2R_df_test['label']


In [313]:
#lay ten cac cot:

colNames_train = list(X_DoS)
labelNames_train = list(Y_DoS)

colNames_test = list(X_DoS_test)

print(colNames_train)
print(labelNames_train[0:10])

['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp', 'Service_IRC', 'Service_X11', 'Service_Z39_50', 'Service_aol', 'Service_auth', 'Service_bgp', 'Service_courier', 'Service_csnet_ns', 'Service_ctf', 'Service_daytime', 'Service_discard', 'Service_domain', 'Service_domain_u', 'Service_echo', 'Se

In [314]:
from sklearn import preprocessing

#train
scaler1 = preprocessing.StandardScaler().fit(X_DoS)
X_DoS = scaler1.transform(X_DoS)

scaler2 = preprocessing.StandardScaler().fit(X_Probe)
X_Probe = scaler2.transform(X_Probe)

scaler3 = preprocessing.StandardScaler().fit(X_R2L)
X_R2L = scaler3.transform(X_R2L)

scaler4 = preprocessing.StandardScaler().fit(X_U2R)
X_U2R = scaler4.transform(X_U2R)

#test

scaler_test1 = preprocessing.StandardScaler().fit(X_DoS_test)
X_DoS_test = scaler_test1.transform(X_DoS_test)

scaler_test2 = preprocessing.StandardScaler().fit(X_Probe_test)
X_Probe_test = scaler_test2.transform(X_Probe_test)

scaler_test3 = preprocessing.StandardScaler().fit(X_R2L_test)
X_R2L_test = scaler_test3.transform(X_R2L_test)

scaler_test4 = preprocessing.StandardScaler().fit(X_U2R_test)
X_U2R_test = scaler_test4.transform(X_U2R_test)


# colNames_train = list(X_DoS)
# colNames_train


# Random Forest All Feature

In [315]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

#all features:
clf_DoS = RandomForestClassifier(n_estimators=50, n_jobs= 2)
clf_Probe = RandomForestClassifier(n_estimators=50, n_jobs= 2)
clf_R2L = RandomForestClassifier(n_estimators=10, n_jobs= 2)
clf_U2R = RandomForestClassifier(n_estimators=10, n_jobs= 2)

#build Models:
clf_DoS.fit(X_DoS, Y_DoS.astype(int))
clf_Probe.fit(X_Probe, Y_Probe.astype(int))
clf_R2L.fit(X_R2L, Y_R2L.astype(int))
clf_U2R.fit(X_U2R, Y_U2R.astype(int))



RandomForestClassifier(n_estimators=10, n_jobs=2)

**Predict**

In [316]:
#DoS:

# clf_DoS.predict(X_DoS_test)
# clf_DoS.predict_proba(X_DoS_test)[0:10]

Y_DoS_pred = clf_DoS.predict(X_DoS_test)

pd.crosstab(Y_DoS_test, Y_DoS_pred, rownames=['Truth'], colnames=['Predicted'], margins=True)

from sklearn.model_selection import cross_val_score
from sklearn import metrics

accuracy_DoS = cross_val_score(clf_DoS, X_DoS_test, Y_DoS_test, cv=10, scoring='accuracy')
print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy_DoS.mean(), accuracy_DoS.std() * 2))

precision_DoS = cross_val_score(clf_DoS, X_DoS_test, Y_DoS_test, cv=10, scoring='precision')
print("Precision: %0.5f (+/- %0.5f)" % (precision_DoS.mean(), precision_DoS.std() * 2))

recall_DoS = cross_val_score(clf_DoS, X_DoS_test, Y_DoS_test, cv=10, scoring='recall')
print("Recall: %0.5f (+/- %0.5f)" % (recall_DoS.mean(), recall_DoS.std() * 2))

f_DoS = cross_val_score(clf_DoS, X_DoS_test, Y_DoS_test, cv=10, scoring='f1')
print("F1: %0.5f (+/- %0.5f)" % (f_DoS.mean(), f_DoS.std() * 2))



Accuracy: 0.99959 (+/- 0.00054)
Precision: 0.99964 (+/- 0.00097)
Recall: 0.99957 (+/- 0.00096)
F1: 0.99960 (+/- 0.00060)


In [317]:
#probe:
Y_Probe_pred = clf_Probe.predict(X_Probe_test)

pd.crosstab(Y_Probe_test, Y_Probe_pred, rownames=['Truth'], colnames=['Predicted'], margins=True)

accuracy_Probe = cross_val_score(clf_Probe, X_Probe_test, Y_Probe_test, cv=10, scoring='accuracy')
print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy_Probe.mean(), accuracy_Probe.std() * 2))

precision_Probe = cross_val_score(clf_Probe, X_Probe_test, Y_Probe_test, cv=10, scoring='precision_macro')
print("Precision: %0.5f (+/- %0.5f)" % (precision_Probe.mean(), precision_Probe.std() * 2))

recall_Probe = cross_val_score(clf_Probe, X_Probe_test, Y_Probe_test, cv=10, scoring='recall_macro')
print("Recall: %0.5f (+/- %0.5f)" % (recall_Probe.mean(), recall_Probe.std() * 2))

f_Probe = cross_val_score(clf_Probe, X_Probe_test, Y_Probe_test, cv=10, scoring='f1_macro')
print("F1: %0.5f (+/- %0.5f)" % (f_Probe.mean(), f_Probe.std() * 2))


Accuracy: 0.99877 (+/- 0.00175)
Precision: 0.99878 (+/- 0.00183)
Recall: 0.99633 (+/- 0.00603)
F1: 0.99739 (+/- 0.00366)


In [318]:
#R2L:
Y_R2L_pred = clf_R2L.predict(X_R2L_test)

pd.crosstab(Y_R2L_test, Y_R2L_pred, rownames=['Truth'], colnames=['Predicted'], margins=True)

accuracy_R2L = cross_val_score(clf_R2L, X_R2L_test, Y_R2L_test, cv=10, scoring='accuracy')
print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy_R2L.mean(), accuracy_R2L.std() * 2))

precision_R2L = cross_val_score(clf_R2L, X_R2L_test, Y_R2L_test, cv=10, scoring='precision_macro')
print("Precision: %0.5f (+/- %0.5f)" % (precision_R2L.mean(), precision_R2L.std() * 2))

recall_R2L = cross_val_score(clf_R2L, X_R2L_test, Y_R2L_test, cv=10, scoring='recall_macro')
print("Recall: %0.5f (+/- %0.5f)" % (recall_R2L.mean(), recall_R2L.std() * 2))

f_R2L = cross_val_score(clf_R2L, X_R2L_test, Y_R2L_test, cv=10, scoring='f1_macro')
print("F1: %0.5f (+/- %0.5f)" % (f_R2L.mean(), f_R2L.std() * 2))


Accuracy: 0.99848 (+/- 0.00177)
Precision: 0.98965 (+/- 0.02062)
Recall: 0.95581 (+/- 0.05832)
F1: 0.96976 (+/- 0.03959)


In [324]:
#U2R
Y_U2R_pred = clf_U2R.predict(X_U2R_test)

pd.crosstab(Y_U2R_test, Y_U2R_pred, rownames=['Truth'], colnames=['Predicted'], margins=True)

accuracy_U2R = cross_val_score(clf_U2R, X_U2R_test, Y_U2R_test, cv=10, scoring='accuracy')
print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy_U2R.mean(), accuracy_U2R.std() * 2))

# precision_U2R = cross_val_score(clf_U2R, X_U2R_test, Y_U2R_test, cv=10, scoring='precision_macro')
# print("Precision: %0.5f (+/- %0.5f)" % (precision_U2R.mean(), precision_U2R.std() * 2))

# recall_U2R = cross_val_score(clf_U2R, X_U2R_test, Y_U2R_test, cv=10, scoring='recall_macro', zero_division=1)
# print("Recall: %0.5f (+/- %0.5f)" % (recall_U2R.mean(), recall_U2R.std() * 2))

# f_U2R = cross_val_score(clf_U2R, X_U2R_test, Y_U2R_test, cv=10, scoring='f1_macro', zero_division=1)
# print("F1: %0.5f (+/- %0.5f)" % (f_U2R.mean(), f_U2R.std() * 2))


Accuracy: 0.99926 (+/- 0.00050)


# Random Forest: feature selected

In [320]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

#set number of tree in random forest = 50
clf = RandomForestClassifier(n_estimators=50, n_jobs= 2)

rfe = RFE(estimator= clf, n_features_to_select=13, step = 1)






**Tim 13 thuoc tinh de chon (neu can)**

In [321]:
#DoS_train:
rfe.fit(X_DoS, Y_DoS.astype(int))
X_rfeDoS = rfe.transform(X_DoS)
true = rfe.support_
rfecolindex_DoS = [i for i, x in enumerate(true) if x]
rfecolname_DoS = list(colNames_train[i] for i in rfecolindex_DoS)

print("Features selected for DoS:", rfecolname_DoS) 

Features selected for DoS: ['src_bytes', 'dst_bytes', 'wrong_fragment', 'num_compromised', 'count', 'srv_count', 'serror_rate', 'same_srv_rate', 'dst_host_count', 'dst_host_same_srv_rate', 'dst_host_srv_serror_rate', 'Service_ecr_i', 'Flag_S0']


In [ ]:
#Probe_train:
rfe.fit(X_Probe, Y_Probe.astype(int))
X_rfeProbe = rfe.transform(X_Probe)
true = rfe.support_
rfecolindex_Probe = [i for i, x in enumerate(true) if x]
rfecolname_Probe = list(colNames_train[i] for i in rfecolindex_Probe)

print("Features selected for Probe:", rfecolname_Probe)

In [ ]:
#R2L_train:
rfe.fit(X_R2L, Y_R2L.astype(int))
X_rfeR2L = rfe.transform(X_R2L)
true = rfe.support_
rfecolindex_R2L = [i for i, x in enumerate(true) if x]
rfecolname_R2L = list(colNames_train[i] for i in rfecolindex_R2L)

print("Features selected for R2L:", rfecolname_R2L)


Features selected for R2L: ['duration', 'src_bytes', 'dst_bytes', 'hot', 'is_guest_login', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'Service_ftp', 'Service_ftp_data']


In [ ]:
#U2R_train:
rfe.fit(X_U2R, Y_U2R.astype(int))
X_rfeU2R = rfe.transform(X_U2R)
true = rfe.support_
rfecolindex_U2R = [i for i, x in enumerate(true) if x]
rfecolname_U2R = list(colNames_train[i] for i in rfecolindex_U2R)

print("Features selected for U2R:", rfecolname_U2R)

Features selected for U2R: ['duration', 'src_bytes', 'dst_bytes', 'hot', 'num_compromised', 'root_shell', 'num_file_creations', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'Service_ftp_data']


In [ ]:
print(X_rfeDoS.shape)
print(X_rfeProbe.shape)
print(X_rfeR2L.shape)
print(X_rfeU2R.shape)

(79306, 13)
(55378, 13)
(47889, 13)
(47257, 13)
